# Read in LES Data and Experiment with comparing to Model output

In [2]:
# activate 
using Pkg
Pkg.activate(".")
using YAML 
import ClimaAtmos as CA
using Revise

  Activating project at `/central/groups/esm/jschmitt/ClimaAtmos.jl/calibration/experiments/resolution_exp`


In [10]:
includet("get_les_metadata.jl")
includet("helper_funcs.jl")
includet("observation_map.jl")

In [4]:
# load in the 3 test sites
les_17 = get_stats_path(get_cfsite_les_dir(17))
les_23 = get_stats_path(get_cfsite_les_dir(23))
les_30 = get_stats_path(get_cfsite_les_dir(30))

"../../../../../zhaoyi/GCMForcedLES/cfsite/07/HadGEM2-A/amip/Output.cfsite30_HadGEM2-A_amip_2004-2008.07.4x/stats/Stats.cfsite30_HadGEM2-A_amip_2004-2008.07.nc"

In [5]:
atmos_config = CA.AtmosConfig(YAML.load_file("prognostic_edmf.yml"))

┌ Warning: The configuration passed to ClimaAtmos contains unused keys: dt_save_to_disk
└ @ ClimaAtmos /central/groups/esm/jschmitt/ClimaAtmos.jl/src/solver/yaml_helper.jl:92
[ Info: Running on CPUSingleThreaded.
[ Info: Setting up single-process ClimaAtmos run
[ Info: Running ClimaCore in unthreaded mode.
┌ Warning: `job_id` is empty and likely not passed to AtmosConfig.
└ @ ClimaAtmos /central/groups/esm/jschmitt/ClimaAtmos.jl/src/solver/types.jl:642
┌ Info: Making AtmosConfig with config files: 
└    /central/groups/esm/jschmitt/ClimaAtmos.jl/src/solver/../../config/default_configs/default_config.yml


ClimaAtmos.AtmosConfig{Float32, ClimaParams.ParamDict{Float32}, Dict{String, Any}, ClimaComms.SingletonCommsContext{ClimaComms.CPUSingleThreaded}, Vector{String}}(ClimaParams.ParamDict{Float32}(Dict{String, Any}("optics_lookup_temperature_min" => Dict{String, Any}("value" => 160, "type" => "float", "description" => "Miminum temperature in the lookup table for optical properties in RRTMGP"), "f_plane_coriolis_frequency" => Dict{String, Any}("value" => 0, "type" => "float"), "Heymsfield_mu_cutoff" => Dict{String, Any}("value" => 6, "type" => "float", "description" => "Limiter for shape parameter mu for ice in P3 scheme. See eq 3 in Morrison and Milbrandt 2015. Units: [-]"), "canopy_emissivity" => Dict{String, Any}("value" => 0.97, "type" => "float", "description" => "Emissivity of the canopy"), "coefficient_a_m_cheng" => Dict{String, Any}("value" => 6.1, "type" => "float", "description" => "a_m for Cheng momentum universal functions. From Cheng et al, 2005. DOI: 10.1007/s10546-004-1425-4

In [6]:
zc_model = get_z_grid(atmos_config, z_max = 40e3)
zc_model

┌ Info: Topography
└   topography = "NoWarp"
┌ Warning: perturb_initstate flag is ignored for single column configuration
└ @ ClimaAtmos /central/groups/esm/jschmitt/ClimaAtmos.jl/src/solver/type_getters.jl:200
┌ Warning: Bubble correction not compatible with single column configuration. It will be switched off.
└ @ ClimaAtmos /central/groups/esm/jschmitt/ClimaAtmos.jl/src/solver/type_getters.jl:211
┌ Info: z heights
│   z_mesh.faces =
│    61-element Vector{ClimaCore.Geometry.ZPoint{Float32}}:
│     ZPoint(0.0)
│     ZPoint(30.0)
│     ZPoint(62.88452)
│     ZPoint(98.928185)
│     ⋮
│     ZPoint(36278.35)
│     ZPoint(38135.195)
└     ZPoint(40000.0)
┌ Info: Resolution stats: 
│   Nq = 1
│   h_elem = 6
│   z_elem = 60
│   ncols = 1
└   ndofs_total = 60


60-element Vector{Float64}:
    15.0
    46.4422607421875
    80.90635681152344
   118.67970275878906
   160.07644653320312
   205.43984985351562
   255.14479064941406
   309.60052490234375
   369.25372314453125
   434.5913391113281
     ⋮
 24915.771484375
 26546.9296875
 28229.1953125
 29957.484375
 31726.03515625
 33528.4609375
 35357.828125
 37206.7734375
 39067.59765625

In [7]:
ref_paths, _ = get_les_calibration_library()

(["../../../../../zhaoyi/GCMForcedLES/cfsite/07/HadGEM2-A/amip/Output.cfsite17_HadGEM2-A_amip_2004-2008.07.4x/stats/Stats.cfsite17_HadGEM2-A_amip_2004-2008.07.nc", "../../../../../zhaoyi/GCMForcedLES/cfsite/07/HadGEM2-A/amip/Output.cfsite23_HadGEM2-A_amip_2004-2008.07.4x/stats/Stats.cfsite23_HadGEM2-A_amip_2004-2008.07.nc"], (17, 23))

In [9]:
# obs_test = get_obs(ref_paths[1], ["ta", "clw"], zc_model)

obs_test = get_obs(
    ref_paths[1],        # filename
    ["thetaa"],       # y_names
    zc_model,            # z_scm
    ti=5.5 * 3600 * 24,  # optional ti argument (default already matches)
    tf=6.0 * 3600 * 24,  # optional tf argument (default already matches)
    model_error=nothing, # optional model_error argument (if none, pass nothing)
    norm_factors_dict=nothing,  # optional norm_factors_dict
    z_score_norm=false,   # boolean flag
    log_vars=[],         # default empty array
    Σ_const=Dict(),     # Σ_const (Dict or nothing)
    Σ_scaling="const"    # Σ_scaling (default string)
)

KeyError: KeyError: key "theta_mean" not found

In [23]:
test_ds = NCDataset(ref_paths[1])
test_ds

ArgumentError: ArgumentError: Package CommonDataModel not found in current path.
- Run `import Pkg; Pkg.add("CommonDataModel")` to install the CommonDataModel package.

In [ ]:
Pkg.add("CommonDataModel")

   Resolving package versions...
    Updating `/central/groups/esm/jschmitt/ClimaAtmos.jl/calibration/experiments/resolution_exp/Project.toml`
  [1fbeeb36] + CommonDataModel v0.3.6
  No Changes to `/central/groups/esm/jschmitt/ClimaAtmos.jl/calibration/experiments/resolution_exp/Manifest.toml`
Precompiling project...
  ✓ StructArrays → StructArraysGPUArraysCoreExt
  ✓ Xorg_xcb_util_jll
  ✓ Libtiff_jll
  ✓ Dbus_jll
  ✓ Wayland_jll
  ✓ eudev_jll
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXinerama_jll
  ✓ Xorg_libXrandr_jll
  ✓ Libglvnd_jll
  ✗ FFMPEG
  ✓ ImageIO
  ✓ libinput_jll
  ✓ Pango_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Xorg_libXcursor_jll
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_libXi_jll
  ✓ Xorg_xkbcomp_jll
  ✓ Xorg_xcb_util_cursor_jll
  ✓ Latexify → DataFramesExt
  ✓ Xorg_xkeyboard_config_jll
  ✓ KernelDensity
  ✓ xkbcommon_jll
  ✓ Cairo
  ✓ Latexify → SparseArraysExt
  ✓ PlotThemes
  ✓ Vulkan_Loader_jll
  ✓ 